In [30]:
import requests
from time import sleep
import numpy as np
import re

# Initialize API session
s = requests.Session()
s.headers.update({'X-API-key': '9EN88DGS'})  # Use your actual API Key

# Constants
TICKERS = ["TP", "AS", "BA"]
MAX_NET_POSITION = 33333 # 50000
MAX_GROSS_POSITION = 100000
ORDER_SIZE = 10000  # Max allowed order size

EPS = {"TP":[0.4,0.33,0.33,0.37],"AS":[0.35,0.45,0.5,0.25],"BA":[0.15,0.5,0.6,0.25]}
OWNERSHIP = {"TP":0.5,"AS":0.5,"BA":0.5} # Changes

def get_tick():
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick'], case['status']
    
# Fetch Market Data
def get_bid_ask(ticker):
    resp = s.get("http://localhost:9999/v1/securities/book", params={"ticker": ticker})
    best_bid = None
    best_ask = None
    while best_bid == None or best_ask == None:
        if resp.ok:
            book = resp.json()
            best_bid = book["bids"][0]["price"] if book["bids"] else None
            best_ask = book["asks"][0]["price"] if book["asks"] else None
        #sleep(0.5)
        #print("stuck in bid ask")
    #print("free")
    return best_bid, best_ask
    #return None, None

# Fetch Position Data
def get_position(ticker):
    resp = s.get("http://localhost:9999/v1/securities")
    if resp.ok:
        for stock in resp.json():
            if stock["ticker"] == ticker:
                return stock["position"]
    return 0


def get_news(own,eps,prev_size):
    resp = s.get ('http://localhost:9999/v1/news', params = {'limit': 50}) # default limit is 20
    if resp.ok:
        news_query = resp.json()
        if len(news_query) > prev_size:
            #print(len(news_query) - prev_size)
            for index in range(len(news_query) - prev_size -1, -1, -1):
                #print(news_query[index]["headline"])
                if "Earnings Estimates" in news_query[index]["headline"]:
                    eps_values = [float(match.group().split("$")[1]) for match in re.finditer(r'Q\d:\s\$\d\.\d{2}', news_query[index]["body"])]

                    # Extract quarter index and ticker
                    quarter_match = re.search(r'#(\d+)', news_query[index]["headline"])
                    ticker = news_query[index]["headline"].split()[0]

                    if quarter_match and ticker in eps:
                        quarter_index = int(quarter_match.group(1)) - 1  # Convert to zero-based index
                        # Ensure we don't go out of bounds
                        for i, new_value in enumerate(eps_values):
                            if quarter_index + i < len(eps[ticker]):
                                eps[ticker][quarter_index + i] = new_value
                    #print("Updated EPS Table")
                    #print(eps)

                elif "Earnings release" in news_query[index]["headline"]:
                    parse = news_query[index]["body"].split("<br>")
                    for eps_news in parse:
                        news = eps_news.split()
                        eps[news[0]][int(news[1].replace("Q","").replace(":",""))-1] = float(news[-1].replace("$",""))
                    #print("Updated EPS Table")
                    #print(eps)

                elif "institution" in news_query[index]["headline"]:
                    ticker = re.search(r'[A-Z]{2}',news_query[index]["body"])
                    value = re.search(r'\d{1,2}\.\d{1,2}\%',news_query[index]["body"])
                    own[ticker.group(0)] = float(value.group(0).replace("%",""))
                    #print("Updated Ownership Table")
                    #print(own)
                    
            return own, eps, len(news_query)
        else:
            return own, eps, prev_size



def get_tp_val(OWNERSHIP, EPS, OWNERSHIP_ERROR, EPS_ERROR):
    TP_eps = sum(EPS['TP'])
    TP_ownership = OWNERSHIP['TP']
    # Midpoint
    TP_g = (TP_eps / 1.43) - 1
    TP_div = TP_eps * 0.80
    TP_DDM = ((TP_div * (1 + TP_g)) / (0.05 - TP_g)) * (1 - ((1 + TP_g) / (1 + 0.05))**5 ) + ((TP_div * ((1 + TP_g)**5) * (1 + 0.02)) / (0.05 - 0.02)) / (1 + 0.05)**5
    TP_pe = TP_eps * 12
    TP_val = (TP_ownership / 100) * TP_DDM + (1 - (TP_ownership / 100)) * TP_pe    
    
    # Min
    min_eps = sum(np.add(EPS['TP'], -1 * EPS_ERROR))
    TP_g = (min_eps / 1.43) - 1
    TP_div = min_eps * 0.80
    TP_DDM = ((TP_div * (1 + TP_g)) / (0.05 - TP_g)) * (1 - ((1 + TP_g) / (1 + 0.05))**5 ) + ((TP_div * ((1 + TP_g)**5) * (1 + 0.02)) / (0.05 - 0.02)) / (1 + 0.05)**5
    TP_pe = min_eps * 12
    if TP_DDM < TP_pe:
        TP_ownership_min = TP_ownership * (1 + OWNERSHIP_ERROR)
    else: 
        TP_ownership_min = TP_ownership * (1 - OWNERSHIP_ERROR)
    tp_min = (TP_ownership_min / 100) * TP_DDM + (1 - (TP_ownership_min / 100)) * TP_pe

    # Max
    max_eps = sum(np.add(EPS['TP'], EPS_ERROR))
    TP_g = (max_eps / 1.43) - 1
    TP_div = max_eps * 0.80
    TP_DDM = ((TP_div * (1 + TP_g)) / (0.05 - TP_g)) * (1 - ((1 + TP_g) / (1 + 0.05))**5 ) + ((TP_div * ((1 + TP_g)**5) * (1 + 0.02)) / (0.05 - 0.02)) / (1 + 0.05)**5
    TP_pe = max_eps * 12
    if TP_DDM < TP_pe:
        TP_ownership_max = TP_ownership * (1 - OWNERSHIP_ERROR)
    else: 
        TP_ownership_max = TP_ownership * (1 + OWNERSHIP_ERROR)
    tp_max = (TP_ownership_max / 100) * TP_DDM + (1 - (TP_ownership_max / 100)) * TP_pe
    
    return round(TP_val, 2), round(tp_min, 2), round(tp_max,2)



def get_as_val(OWNERSHIP, EPS, OWNERSHIP_ERROR, EPS_ERROR):
    AS_eps = sum(EPS['AS'])
    AS_ownership = OWNERSHIP['AS']
    # Midpoint
    AS_g = (AS_eps / 1.55) - 1
    AS_div = AS_eps * 0.50
    AS_DDM = ((AS_div * (1 + AS_g)) / (0.075 - AS_g)) * (1 - ((1 + AS_g) / (1 + 0.075))**5 ) + ((AS_div * ((1 + AS_g)**5) * (1 + 0.02)) / (0.075 - 0.02)) / (1 + 0.075)**5
    AS_pe = AS_eps * 16
    AS_val = (AS_ownership / 100) * AS_DDM + (1 - (AS_ownership / 100)) * AS_pe
    
    # Min
    min_eps = sum(np.add(EPS['AS'], -1 * EPS_ERROR))
    AS_g = (min_eps / 1.55) - 1
    AS_div = min_eps * 0.50
    AS_DDM = ((AS_div * (1 + AS_g)) / (0.075 - AS_g)) * (1 - ((1 + AS_g) / (1 + 0.075))**5 ) + ((AS_div * ((1 + AS_g)**5) * (1 + 0.02)) / (0.075 - 0.02)) / (1 + 0.075)**5
    AS_pe = min_eps * 16
    if AS_DDM < AS_pe:
        AS_ownership_min = AS_ownership * (1 + OWNERSHIP_ERROR)
    else: 
        AS_ownership_min = AS_ownership * (1 - OWNERSHIP_ERROR)
    as_min = (AS_ownership_min / 100) * AS_DDM + (1 - (AS_ownership_min / 100)) * AS_pe

    # Max
    max_eps = sum(np.add(EPS['AS'], EPS_ERROR))
    AS_g = (max_eps / 1.55) - 1
    AS_div = max_eps * 0.50
    AS_DDM = ((AS_div * (1 + AS_g)) / (0.075 - AS_g)) * (1 - ((1 + AS_g) / (1 + 0.075))**5 ) + ((AS_div * ((1 + AS_g)**5) * (1 + 0.02)) / (0.075 - 0.02)) / (1 + 0.075)**5
    AS_pe = max_eps * 16
    if AS_DDM < AS_pe: 
        AS_ownership_max = AS_ownership * (1 - OWNERSHIP_ERROR)
    else: 
        AS_ownership_max = AS_ownership * (1 + OWNERSHIP_ERROR)
    as_max = (AS_ownership_max / 100) * AS_DDM + (1 - (AS_ownership_max / 100)) * AS_pe
    
    return round(AS_val,2), round(as_min,2), round(as_max,2)



def get_ba_val(OWNERSHIP, EPS, OWNERSHIP_ERROR, EPS_ERROR):
    BA_eps = sum(EPS['BA'])
    BA_ownership = OWNERSHIP['BA']
    # Midpoint
    BA_g = (BA_eps / 1.50) - 1
    BA_pe_inst = 20 * (1 + BA_g) * BA_eps
    BA_pe_retail = BA_eps * 20
    BA_val = (BA_ownership / 100) * BA_pe_inst + (1 - (BA_ownership / 100)) * BA_pe_retail
    
    # Min
    min_eps = sum(np.add(EPS['BA'], -1 * EPS_ERROR))
    BA_g = (min_eps / 1.50) - 1
    BA_pe_inst = 20 * (1 + BA_g) * min_eps
    BA_pe_retail = min_eps * 20
    if BA_pe_inst < BA_pe_retail: 
        BA_ownership_min = BA_ownership * (1 + OWNERSHIP_ERROR)
    else: 
        BA_ownership_min = BA_ownership * (1 - OWNERSHIP_ERROR)
    ba_min = (BA_ownership_min / 100) * BA_pe_inst + (1 - (BA_ownership_min / 100)) * BA_pe_retail

    # Max
    max_eps = sum(np.add(EPS['BA'], EPS_ERROR))
    BA_g = (max_eps / 1.50) - 1
    BA_pe_inst = 20 * (1 + BA_g) * max_eps
    BA_pe_retail = max_eps * 20
    if BA_pe_inst < BA_pe_retail: 
        BA_ownership_max = BA_ownership * (1 - OWNERSHIP_ERROR)
    else: 
        BA_ownership_max = BA_ownership * (1 + OWNERSHIP_ERROR)
    ba_max = (BA_ownership_max / 100) * BA_pe_inst + (1 - (BA_ownership_max / 100)) * BA_pe_retail
    
    return round(BA_val,2), round(ba_min,2), round(ba_max,2)



if __name__ == "__main__":
    tick, status = get_tick()
    default_size = 0

    # Initializing for triangulation
    tp_min_old = 0
    as_min_old = 0
    ba_min_old = 0
    tp_max_old = 10000
    as_max_old = 10000
    ba_max_old = 10000
    quarter = 0
    size_percentage = 1/8
    threshold = 1

    # Initial error
    OWNERSHIP_ERROR = {"TP":0.25,"AS":0.30,"BA":0.35} # Added 5% to the initial values, decreases by 5% each quarter
    EPS_ERROR = {"TP":[0.02, 0.04, 0.06, 0.08],"AS":[0.04, 0.08, 0.12, 0.16],"BA":[0.06, 0.12, 0.18, 0.24]}
    OWNERSHIP_OLD = {"TP":0.5,"AS":0.5,"BA":0.5}
    EPS_OLD = {"TP":[0.4,0.33,0.33,0.37],"AS":[0.35,0.45,0.5,0.25],"BA":[0.15,0.5,0.6,0.25]}

    taking_position_tp = True
    taking_position_as = True
    taking_position_ba = True

    while status == "ACTIVE":

        # Get news, update values if the news changes our valuations
        OWNERSHIP, EPS, default_size = get_news(OWNERSHIP, EPS,default_size)
        if OWNERSHIP != OWNERSHIP_OLD or EPS != EPS_OLD:
            tp_val, tp_min, tp_max = get_tp_val(OWNERSHIP, EPS, np.array(OWNERSHIP_ERROR['TP']), np.array(EPS_ERROR['TP']))
            as_val, as_min, as_max = get_as_val(OWNERSHIP, EPS, np.array(OWNERSHIP_ERROR['AS']), np.array(EPS_ERROR['AS']))
            ba_val, ba_min, ba_max = get_ba_val(OWNERSHIP, EPS, np.array(OWNERSHIP_ERROR['BA']), np.array(EPS_ERROR['BA']))
            OWNERSHIP_OLD = OWNERSHIP
            EPS_OLD = EPS
            taking_position_tp == True
            taking_position_as == True
            taking_position_ba == True
            print("News Updated")

        # Triangulation - setting closer min/max values
        if tp_min > tp_min_old:
            tp_min_old = tp_min
        if as_min > as_min_old:
            as_min_old = as_min
        if ba_min > ba_min_old:
            ba_min_old = ba_min
        if tp_max < tp_max_old:
            tp_max_old = tp_max
        if as_max < as_max_old:
            as_max_old = as_max
        if ba_max < ba_max_old:
            ba_max_old = ba_max
        

        # Trading logic: We buy at ask price, sell at bid price
        tp_bid, tp_ask = get_bid_ask("TP")  
        as_bid, as_ask = get_bid_ask("AS")  
        ba_bid, ba_ask = get_bid_ask("BA")  

        if tp_bid < tp_min_old or tp_bid > tp_max_old or tp_ask < tp_min_old or tp_ask > tp_max_old:
            tp_min_old = tp_min
            tp_max_old = tp_max
        if as_bid < as_min_old or as_bid > as_max_old or as_ask < as_min_old or as_ask > as_max_old:
            as_min_old = as_min
            as_max_old = as_max
        if ba_bid < ba_min_old or ba_bid > ba_max_old or ba_ask < ba_min_old or ba_ask > ba_max_old:
            ba_min_old = ba_min
            ba_max_old = ba_max

       
        # If taking a position, buy/sell
        # Buy
        if get_position("TP") + get_position("AS") + get_position("BA") < MAX_GROSS_POSITION:
            if tp_val > (tp_ask + threshold) and get_position("TP") < MAX_NET_POSITION and taking_position_tp == True:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'TP', 'type': 'MARKET', 'quantity': min(ORDER_SIZE * size_percentage, MAX_NET_POSITION - get_position("TP")), 'action': 'BUY'})
            if as_val > (as_ask + threshold) and get_position("AS") < MAX_NET_POSITION and taking_position_as == True:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'AS', 'type': 'MARKET', 'quantity': min(ORDER_SIZE * size_percentage, MAX_NET_POSITION - get_position("AS")), 'action': 'BUY'})
            if ba_val > (ba_ask + threshold) and get_position("BA") < MAX_NET_POSITION and taking_position_ba == True:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'BA', 'type': 'MARKET', 'quantity': min(ORDER_SIZE * size_percentage, MAX_NET_POSITION - get_position("BA")), 'action': 'BUY'})

        # Sell
        if get_position("TP") + get_position("AS") + get_position("BA") > -1 * MAX_GROSS_POSITION:
            if tp_val < (tp_bid - threshold) and get_position("TP") > -1 *  MAX_NET_POSITION and taking_position_tp == True:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'TP', 'type': 'MARKET', 'quantity': min(ORDER_SIZE * size_percentage, abs(-1 * MAX_NET_POSITION - get_position("TP"))), 'action': 'SELL'})
            if as_val < (as_bid - threshold) and get_position("AS") > -1 *  MAX_NET_POSITION and taking_position_as == True:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'AS', 'type': 'MARKET', 'quantity': min(ORDER_SIZE * size_percentage, abs(-1 * MAX_NET_POSITION - get_position("TP"))), 'action': 'SELL'})
            if ba_val < (ba_bid - threshold) and get_position("BA") > -1 *  MAX_NET_POSITION and taking_position_ba == True:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'BA', 'type': 'MARKET', 'quantity': min(ORDER_SIZE * size_percentage, abs(-1 * MAX_NET_POSITION - get_position("TP"))), 'action': 'SELL'})
        

        # If the values hit min/max, rebalance
        if (tp_bid < tp_min_old or tp_bid > tp_max_old) and get_position("TP") > 0: # If we are long, and prices hit min/max, sell all. Stop trading until the news hits again. 
            taking_position = False
            print("TP long reset")
            while get_position("TP") != 0:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'TP', 'type': 'MARKET', 'quantity': min(ORDER_SIZE, get_position("TP")), 'action': 'SELL'})
            
        if (tp_ask < tp_min_old or tp_ask > tp_max_old) and get_position("TP") < 0:  # If we are short, and prices hit min/max, sell all. Stop trading until the news hits again. 
            taking_position = False
            print("TP short reset")
            while get_position("TP") != 0:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'TP', 'type': 'MARKET', 'quantity': min(ORDER_SIZE, get_position("TP")), 'action': 'BUY'})
        
        if (as_bid < as_min_old or as_bid > as_max_old) and get_position("AS") > 0: # If we are long, and prices hit min/max, sell all. Stop trading until the news hits again. 
            taking_position = False
            print("AS long reset")
            while get_position("AS") != 0:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'AS', 'type': 'MARKET', 'quantity': min(ORDER_SIZE, get_position("AS")), 'action': 'SELL'})

        if (as_bid < as_min_old or as_bid > as_max_old) and get_position("AS") < 0:  # If we are short, and prices hit min/max, sell all. Stop trading until the news hits again. 
            taking_position = False
            print("AS short reset")
            while get_position("AS") != 0:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'AS', 'type': 'MARKET', 'quantity': min(ORDER_SIZE, get_position("AS")), 'action': 'BUY'})

        if (ba_bid < ba_min_old or ba_bid > ba_max_old) and get_position("BA") > 0: # If we are long, and prices hit min/max, sell all. Stop trading until the news hits again. 
            taking_position = False
            print("BA long reset")
            while get_position("BA") != 0:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'BA', 'type': 'MARKET', 'quantity': min(ORDER_SIZE, get_position("BA")), 'action': 'SELL'})

        if (ba_bid < ba_min_old or ba_bid > ba_max_old) and get_position("BA") < 0:  # If we are short, and prices hit min/max, sell all. Stop trading until the news hits again. 
            taking_position = False
            print("BA short reset")
            while get_position("BA") != 0:
                resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'BA', 'type': 'MARKET', 'quantity': min(ORDER_SIZE, get_position("BA")), 'action': 'BUY'})
            

        # Hardcoding error values, printing estimates and actual bid ask values
        if tick > 60 and tick < 120: #Q1
            OWNERSHIP_ERROR = {"TP":0.2,"AS":0.25,"BA":0.30} 
            EPS_ERROR = {"TP":[0, 0.02, 0.04, 0.06],"AS":[0, 0.04, 0.08, 0.12],"BA":[0, 0.06, 0.12, 0.18]}
            if quarter == 0:
                quarter += 1
                print("Quarter " + str(quarter))
                print(tp_val, tp_min_old, tp_max_old, tp_min, tp_max, get_bid_ask("TP"))
                print(as_val, as_min_old, as_max_old, as_min, as_max, get_bid_ask("AS"))
                print(ba_val, ba_min_old, ba_max_old, ba_min, ba_max, get_bid_ask("BA"))
        if tick > 120 and tick < 180: #Q2
            OWNERSHIP_ERROR = {"TP":0.15,"AS":0.20,"BA":0.25} 
            EPS_ERROR = {"TP":[0, 0, 0.02, 0.04],"AS":[0, 0, 0.04, 0.08],"BA":[0, 0, 0.06, 0.12]}
            if quarter <= 1:
                quarter += 1
                print("Quarter " + str(quarter))
                print(tp_val, tp_min_old, tp_max_old, tp_min, tp_max, get_bid_ask("TP"))
                print(as_val, as_min_old, as_max_old, as_min, as_max, get_bid_ask("AS"))
                print(ba_val, ba_min_old, ba_max_old, ba_min, ba_max, get_bid_ask("BA"))
        if tick > 180 and tick < 240: #Q3
            OWNERSHIP_ERROR = {"TP":0.10,"AS":0.15,"BA":0.20} 
            EPS_ERROR = {"TP":[0, 0, 0, 0.02],"AS":[0, 0, 0, 0.04],"BA":[0, 0, 0, 0.06]}
            if quarter <= 2:
                quarter += 1
                print("Quarter " + str(quarter))
                print(tp_val, tp_min_old, tp_max_old, tp_min, tp_max, get_bid_ask("TP"))
                print(as_val, as_min_old, as_max_old, as_min, as_max, get_bid_ask("AS"))
                print(ba_val, ba_min_old, ba_max_old, ba_min, ba_max, get_bid_ask("BA"))
        if tick > 240: #Q4
            OWNERSHIP_ERROR = {"TP":0.05,"AS":0.10,"BA":0.15} 
            EPS_ERROR = {"TP":[0, 0, 0, 0],"AS":[0, 0, 0, 0],"BA":[0, 0, 0, 0]}
            if quarter <= 3:
                quarter += 1
                print("Quarter " + str(quarter))
                print(tp_val, tp_min_old, tp_max_old, tp_min, tp_max, get_bid_ask("TP"))
                print(as_val, as_min_old, as_max_old, as_min, as_max, get_bid_ask("AS"))
                print(ba_val, ba_min_old, ba_max_old, ba_min, ba_max, get_bid_ask("BA"))

        sleep(0.5)
        tick, status = get_tick()

News Updated
Quarter 1
26.7 16.06 51.66 16.06 51.66 (26.41, 26.45)
13.93 4.72 31.55 4.72 31.55 (17.8, 18.25)
30.0 14.52 50.12 14.52 50.12 (36.62, 36.67)
Quarter 2
26.7 16.06 51.66 16.06 51.66 (26.41, 26.45)
13.93 4.72 31.55 4.72 31.55 (17.8, 18.25)
30.0 14.52 50.12 14.52 50.12 (36.29, 36.67)
Quarter 3
26.7 16.06 51.66 16.06 51.66 (26.41, 26.45)
13.93 4.72 31.55 4.72 31.55 (18.2, 18.25)
30.0 14.52 50.12 14.52 50.12 (36.16, 36.23)
Quarter 4
26.7 16.06 51.66 16.06 51.66 (22.8, 22.81)
13.93 4.72 31.55 4.72 31.55 (17.41, 17.42)
30.0 14.52 50.12 14.52 50.12 (31.55, 31.62)
